Author: Shawn Whitfield <br>
Date: 2022-10-14 <br>
Version: 1

## Specifications

Implement an ipywidget that allows you to flip through all of the events, for every game of a 
given season, with the ability to switch between the regular season and playoffs. Draw the event 
coordinates on the provided ice rink image, similar to the example shown below (you can just print 
the event data when there are no coordinates). You may also print whatever information you find useful, 
such as game metadata/boxscores, and event summaries (but this is not required). Take a screenshot 
of the tool and add it to the blog post, accompanied with the code for the tool and a brief 
(1-2 sentences) description of what your tool does. You do not need to worry about embedding 
the tool into the blogpost.

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json

# Imports for JupyterLite
try:
    import piplite
    await piplite.install(['ipywidgets'])
except ImportError:
    pass
import ipywidgets as widgets
from IPython.display import display

## Importing Data and needed files 

In [2]:
def combine_jsons(folder):
    """
    Takes a directory (folder) and reads all of the .json files in the folder into a dictionary.
    Returns a dictionary containing all filenames as keys, the file contents are the values.
    """
    # Make a dictionary to store all the jsons
    all_data = {}
    files_read = []
    # Go through each file in directory
    for filename in os.listdir(folder):
        
        # Parse the filename
        name = filename.split('.json')[0]
        is_json = filename[-5:] == '.json'
        
        # If it's a json:
        if is_json:
            # Open and load file
            file = open(f"{folder}{filename}")
            file_dict = json.load(file)
            files_read.append(name)
            # Add file to dictionary with key as filename (minus .json)
            all_data[name] = file_dict
#     print(f"Files read into dictionary \"data\": {files_read}")
    return all_data

# Read in the image of the rink
rink = plt.imread("nhl_rink.png")

# Load the data
data = combine_jsons('raw_data/')

## Widget-containing class

In [5]:
class Debugger:
    def __init__(self,data):
        
        ##INITIALIZATION VARIABLE##
        self.data = data
        
        ##CORE VARIABLES##
            
        self.current_data = data #dict, contains the current dictionary (may be a pared-down version of data)
        self.current_season = str('2016') #str, gives the current season
        self.current_game_type = 'Regular season' #str, gives the current game type ("Regular season" or "Playoffs")
        self.current_game_id = '2016020532' #str, is the game currently being focused on ; initializes at first position
        self.current_event = 1 #int, is the event currently being focused on
        
        ##DESCRIBER VARIABLES (used in description of event)##
        
        self.away_team = self.current_data[self.current_game_id]['gameData']['teams']['away']['abbreviation']
        self.home_team = self.current_data[self.current_game_id]['gameData']['teams']['home']['abbreviation']
        self.start_time = self.current_data[self.current_game_id]['gameData']['datetime']['dateTime']
        self.end_time = self.current_data[self.current_game_id]['gameData']['datetime']['endDateTime']
        self.final_score = self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][-1]['about']['goals']
        self.coordinates = self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][self.current_event]['coordinates']
        self.event_description = self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][self.current_event]['result']['description']
        self.event_time = f"{self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][self.current_event]['about']['periodTime']} P-{self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][self.current_event]['about']['period']}"
        self.about_event = self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][self.current_event]['about']
        
        ##WIDGETS##
        
        # Controls the output of the cell; makes sure messages don't build up in the cell.
        self.output = widgets.Output()
        
        # Slides to select a particular season
        self.season = widgets.IntSlider(
            value = self.season_range()[0],
            min = self.season_range()[0],
            max = self.season_range()[1],
        )
        self.season.observe(self.update_season,'value')
        
        #Toggles between playoff games and regular season games
        self.game_type = widgets.ToggleButtons(options=['Regular season','Playoffs'])
        self.game_type.observe(self.update_game_type,'value')
        
        # Slides to select a particular game
        self.game_id = widgets.SelectionSlider(
            options = sorted(list(self.current_data.keys())),
            value = self.current_game_id,
            description='game_id',
        #     disabled=False,
        #     continuous_update=False,
        #     orientation='horizontal',
        #     readout=True
        )
        self.game_id.observe(self.update_game_id,'value')

        # Slides to select a particular event in a game
        self.event = widgets.IntSlider(
            value = 1,
            min = 1,
            max = len(self.current_data[self.current_game_id]['liveData']['plays']['allPlays']) ,
            description = 'event'
        )
        self.event.observe(self.update_event,'value')
        
    ##HELPER FUNCTION TO CREATE WIDGETS##
    
    def season_range(self):
        """
        Lists the files (labeled by game_id) in the "data" dictionary and returns a tuple indicating 
        the minimum season and maximum season (both formatted as strings).
        Needed for "season" widget
        """
        min_season = 3000
        max_season = 0
        for game_id in self.data.keys():
            season = int(game_id[:4])
            if season < min_season:
                min_season = season
            if season > max_season:
                max_season = season
        return (str(min_season), str(max_season))
    
    ##FUNCTIONS TO DISPLAY INFORMATION##
    def plot_coordinates(self):
        """
        Takes self.coordinates (if not empty), which is a dictionary of coordinates{'x':x,'y':y} and  
        plots a point at these coordinates on a hockey rink image with a description + event_time as titles.
        """
        if self.coordinates:
            plt.scatter(x = self.coordinates['x'],y = self.coordinates['y'], zorder = 1, c = 'green', marker = 'D', s = 100)
            plt.imshow(rink,zorder=0,extent = [-100,100,-42.5,42.5])
            plt.title(self.event_time,fontsize = 10)
            plt.suptitle(self.event_description, fontsize = 16, y=.90)
            plt.xlabel('feet')
            plt.ylabel('feet')
        #     plt.axis('off')
            plt.show()
    
    def display_info(self):
        """
        Displays select info about the currently selected event.
        """
        print(f"Game id: {self.current_game_id} ;  Game number {self.current_game_id[-4:].lstrip('0')} ; {self.home_team} (home) vs. {self.away_team} (away)")
        print(f"Game start: {self.start_time}, game end: {self.end_time}")
        print(f"Final score: {self.final_score}")
        print(f"Total events: {len(self.current_data[self.current_game_id]['liveData']['plays']['allPlays'])}")
        self.plot_coordinates()
        print(self.event_description)
        print(self.about_event)
  
#For debugging
#     def print_vars(self):
#         print(self.current_data.keys())
#         print(self.current_season) 
#         print(self.current_game_type) 
#         print(self.current_game_id)
#         print(self.current_event)

#For debugging
#     def print_secondary_vars(self):
#         print(self.away_team)
#         print(self.home_team)
#         print(self.start_time)
#         print(self.end_time)
#         print(self.final_score)
#         print(self.coordinates)
#         print(self.event_description) 
#         print(self.event_time)
#         print(self.about_event)
    
    ##FUNCTIONS TO UPDATE VARIABLES/WIDGETS##
    
    def filter_season(self):
        """
        Updates "self.current_data" with only the entries of "data" that fit that season.
        Needed for updating "current_data" to only show particular season info.
        """
        data_in_season = {}
        for game_id in self.data.keys():
            if str(self.current_season) == game_id[:4]:
                data_in_season[game_id] = self.data[game_id]
        self.current_data = data_in_season

    def filter_playoffs(self):
        """
        Updates the "self.current_data" dict to filter by playoffs  (depending on the state of self.game_type).
        Needed for updating "current_data" to only show info for regular season/playoffs.
        """
        data_in_playoffs = {}
        for game_id in self.current_data.keys():
            if self.current_game_type == "Playoffs":
                if game_id[4:6] == '03':
                    data_in_playoffs[game_id] = data[game_id]
            else:
                if game_id[4:6] == '02':
                    data_in_playoffs[game_id] = data[game_id]
        self.current_data = data_in_playoffs
        
    def update_vars(self):
        """
        Catch-all function, updates the descriptive variables used in display_info().
        """
        self.away_team = self.current_data[self.current_game_id]['gameData']['teams']['away']['abbreviation']
        self.home_team = self.current_data[self.current_game_id]['gameData']['teams']['home']['abbreviation']
        self.start_time = self.current_data[self.current_game_id]['gameData']['datetime']['dateTime']
        self.end_time = self.current_data[self.current_game_id]['gameData']['datetime']['endDateTime']
        self.final_score = self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][-1]['about']['goals']
        self.coordinates = self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][self.current_event]['coordinates']
        self.event_description = self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][self.current_event]['result']['description']
        self.event_time = f"{self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][self.current_event]['about']['periodTime']} P-{self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][self.current_event]['about']['period']}"
        self.about_event = self.current_data[self.current_game_id]['liveData']['plays']['allPlays'][self.current_event]['about']
    
    def update_season(self,x):
        """
        Updates self.current_season with the new input from the season widget.
        """
        self.output.clear_output()
        with self.output:
            self.current_season = x.new
            self.filter_season()
            self.filter_playoffs()
            self.game_id.options = sorted(list(self.current_data.keys()))
            self.game_id.value = self.current_game_id
            self.update_vars()
#             self.print_secondary_vars() #for debugging
            self.display_info()
            
    def update_game_type(self,x):
        """
        Updates self.current_game_type with the new input from the game_type widget.
        """
        self.output.clear_output()
        with self.output:
            self.current_game_type = x.new
            self.filter_season()
            self.filter_playoffs()
            self.game_id.options = sorted(list(self.current_data.keys()))
            self.game_id.value = self.current_game_id
            self.update_vars()
#             self.print_secondary_vars() #for debugging
            self.display_info()
    
    def update_game_id(self,x):
        """
        Updates self.current_game_id with the new input from the game_id widget.
        """
        self.output.clear_output()
        with self.output:
            self.current_game_id = x.new
            self.update_vars()
#             self.print_secondary_vars() #for debugging
            self.display_info()
    
    def update_event(self,x):
        """
        Updates self.current_event with the new input from the event widget.
        """
        self.output.clear_output()
        with self.output:
            self.current_event = x.new
            self.update_vars()
#             self.print_secondary_vars() #for debugging
            self.display_info()

    def display_output(self):
        display(self.output)
        

# Running the widget

In [6]:
d = Debugger(data)
display(d.season)
display(d.game_type)
display(d.game_id)
display(d.event)
d.display_output()

IntSlider(value=2016, max=2020, min=2016)

ToggleButtons(options=('Regular season', 'Playoffs'), value='Regular season')

SelectionSlider(description='game_id', index=531, options=('2016020001', '2016020002', '2016020003', '20160200…

IntSlider(value=1, description='event', max=338, min=1)

Output()